In [1]:
import pandas as pd
import math as mt
from sklearn import preprocessing

In [2]:
def predict(row,coeff_values):
#     print(row)
#     print(coeff_values)
    y_predict = coeff_values[0]
    for i in range(len(coeff_values)-1):
        y_predict += row[i]*coeff_values[i+1]
#     print(y_predict)
    return 1.0/(1.0+mt.exp(-y_predict))

#https://www.youtube.com/watch?v=xEa78n3K4gU - very good explaination on how to do Stochastic Gradient Optimization
def sgd(rows,coeffs,learn_rate,epochs):
    coeff_values=[0.0 for i in range(len(coeffs))]
    for e in range(epochs):
#         print(coeff_values)
        for row in range(rows.shape[0]):
            r = rows.iloc[row]
#             print(r)
#             break
            eachrow = [r[val] for val in r.keys()]
#             print(eachrow)
            y_cap = predict(eachrow,coeff_values)
#             print("y_cap\t"+str(y_cap))
            err = eachrow[-1] - y_cap
#             print("err\t"+str(err))
            jump = (learn_rate * err * y_cap * (1.0 - y_cap)) #doing gradient ascent, hence jump here is the climb
#             if jump==0:
#                 print('jump\t'+str(jump))
            coeff_values[0] = coeff_values[0] + jump
            for i in range(len(coeff_values)-1):
                coeff_values[i+1] = coeff_values[i+1] + jump * eachrow[i]
    return coeff_values

def logistic_regression(train,test,learn_rate,epochs,coeffs):
    predicted = []
#     coeff_values=[0.0 for i in range(len(coeffs))]
    coeff = sgd(train, coeffs, learn_rate, epochs)
#     print(coeff)
    for row in range(test.shape[0]):
        new_row = test.iloc[row]
        eachrow = [new_row[val] for val in new_row.keys()]
        y_cap = predict(eachrow,coeff)
        y_cap = round(y_cap)
        predicted.append(y_cap)
    return predicted

def normalize_data(data):
    for val in data.columns:
        data[val] = data[val].astype(float)
    minobj = data.min()
    maxobj = data.max()
    for key in data.iloc[0].keys():
        if key!='class':
            for row in range(data.shape[0]):
#                 print("before\t"+str(data[key][row]))
#                 print("min\t"+str(minobj[key]))
#                 print("max\t"+str(maxobj[key]))
                data[key][row] = (float(data[key][row]) - minobj[key]) / (float(maxobj[key]) - minobj[key])
#                 print("before\t"+str(data[key][row]))
#             break
#     print(data)
#     print(minobj['preg'])
# #     print(maxobj)
    return data
#     x = data.values
#     min_max = preprocessing.MinMaxScaler()
#     x_scaled = min_max.fit_transform(x)
#     normalized = pd.DataFrame(x_scaled)
#     print(normalized)
            

In [3]:
def get_confusion_matrix(actual,predictions):
    TN,TP,FN,FP=0,0,0,0
#     actual = int(actual)
    for i in range(len(actual)):
        if actual[i]==1.0 and predictions[i]==1:
            TP += 1
        elif actual[i]==0.0 and predictions[i]==0:
            TN += 1
        elif actual[i]==1.0 and predictions[i]==0:
            FN += 1
        else:
            FP += 1
    return TN,TP,FN,FP

In [11]:
data = pd.read_csv("Diabetes.csv")
print(data)
coeffs = [val for val in data.columns]
# print(coeffs)
# coeff_values=[0.0 for i in range(len(coeffs))]
# print(coeff_values)
learn_rate = 0.15
epochs = 10
normalized = normalize_data(data)
# print(normalized)
#randomly splitting the dataset into 8:2 train and test
train=normalized.sample(frac=0.8,random_state=100) #random state is a seed value
test=normalized.drop(train.index)
# print(len(predictions))
predictions = logistic_regression(train,test,learn_rate,epochs,coeffs)
actual = list(test['class'])
# print(len(actual))
TN,TP,FN,FP = get_confusion_matrix(actual,predictions)
print("\n\nTrue Positive: "+str(TP))
print("False Positive: "+str(FP))
print("True Negative: "+str(TN))
print("False Negative: "+str(FN))
print("Accuracy: "+str((TP+TN)*100/(TP+TN+FP+FN)))

# print(list(test['class']))
# print(data['preg'][0])

     preg  plas  pres  skin  insu  mass   pedi  age  class
0       5     0    80    32     0  41.0  0.346   37      1
1       4    92    80     0     0  42.2  0.237   29      0
2       4   137    84     0     0  31.2  0.252   30      0
3       3    61    82    28     0  34.4  0.243   46      0
4       1    90    62    12    43  27.2  0.580   24      0
..    ...   ...   ...   ...   ...   ...    ...  ...    ...
763     8    95    72     0     0  36.8  0.485   57      0
764     8   126    88    36   108  38.5  0.349   49      0
765     1   139    46    19    83  28.7  0.654   22      0
766     3   116     0     0     0  23.5  0.187   23      0
767     3    99    62    19    74  21.8  0.279   26      0

[768 rows x 9 columns]


True Positive: 23
False Positive: 12
True Negative: 88
False Negative: 31
Accuracy: 72.07792207792208
